**Use Case ID**: UC00173

**Title**: Real Time Accessible Parking  
**Duration**: 1-2 hours to understand the dataset  
**Level**: Beginner-Intermediate  
**Pre-requisite Skills**:
- Basic Python (Pandas, API requests)
- Data visualisation using Folium
- Understanding of geospatial data (lat/lon)


**Datasets used**:
- [On-street parking bays](https://data.melbourne.vic.gov.au/Transport/on-street-parking-bays)
- [Off-street car parks with capacity and type](https://data.melbourne.vic.gov.au/Transport/off-street-car-parks-with-capacity-and-type)
- [On-street car parking meters with location](https://data.melbourne.vic.gov.au/Transport/on-street-car-parking-meters-with-location)
- [Parking restrictions](https://data.melbourne.vic.gov.au/Transport/on-street-parking-bays)


###  Scenario

#### UC00173_Real_Time_Accessible_Parking

In the City of Melbourne, accessible parking is a critical resource for residents and visitors living with mobility impairments. However, these designated bays are often difficult to locate in real time, and occupancy can vary dramatically depending on time of day, location, and nearby events.

The City of Melbourne has deployed an advanced sensor network that monitors individual parking bays. These sensors, when combined with parking restriction data, provide a powerful foundation to develop a smart, real-time parking availability system targeted specifically at accessible (disabled-only) parking spaces.

This use case proposes a live dashboard that identifies the location and occupancy status of disability bays across the municipality. The project leverages existing open datasets to improve urban mobility, reduce the stress of finding suitable parking, and promote more inclusive city infrastructure.


In [40]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import os
from functools import reduce
import folium
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
from dotenv import load_dotenv
from ipywidgets import Tab, VBox, HTML, Output


# RT ACCESSIBLE PARKING – 1.1

### Visualising On-Street Accessible Parking Bays

This visualisation shows where accessible (disability) parking bays are located across Melbourne streets.

By using open data provided by the City of Melbourne, we can download a list of all known accessible bays and display them on an interactive map. Each marker shows which street it’s on and when it was last updated.

This tool can help residents, planners, and visitors quickly see where accessible parking is available — and identify gaps in coverage.


## Step 1: Environment setup (dotenv)

We install and (if needed) update the `python-dotenv` library to manage environment variables such as API keys securely via a `.env` file.  
The output below is just the installation log from `pip` (e.g. “Requirement already satisfied” or the version installed).  
No data is loaded in this step — data loading begins in **Step 2**.


In [41]:
def fetch_parking_dataset(base_url, dataset_id, api_key=None, limit=100, max_pages=100):
    """
    Downloads and aggregates data from City of Melbourne’s Open Data API using paginated requests.
    
    Parameters:
        base_url (str): Base API URL.
        dataset_id (str): Dataset endpoint name.
        api_key (str): Optional API key. If not provided, will try to use MELBOURNE_API_KEY from environment.
        limit (int): Number of records per page (maximum 100).
        max_pages (int): Maximum number of pages to fetch to prevent infinite loops.
        
    Returns:
        pd.DataFrame: All records combined into a DataFrame.
    """
    api_key = api_key or os.environ.get("MELBOURNE_API_KEY")
    if not api_key:
        raise ValueError("API key is required. Set MELBOURNE_API_KEY in your environment or pass it directly.")

    all_results = []

    for page in range(max_pages):
        offset = page * limit
        url = f"{base_url}{dataset_id}/records?limit={limit}&offset={offset}&api_key={api_key}"

        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            data = response.json().get("results", [])
        except requests.exceptions.RequestException as e:
            print(f"Page {page + 1} failed: {e}")
            break

        if not data:
            break  # No more data to fetch

        all_results.extend(data)

        if len(data) < limit:
            break  # Reached the final page

    return pd.DataFrame(all_results)


# Set up the access details for the data website
load_dotenv()
API_KEY = os.environ.get("MELBOURNE_API_KEY")  # Gets your private API key from your computer
BASE_URL = 'https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/'


## Step 2: Load the Accessible Parking Bay Data

This step tells the program which dataset to get. We’re using the one called `on-street-parking-bays`, which lists all bays on Melbourne streets — including info like the street name, type of parking, and restrictions.

The table preview shows the first few rows.


In [42]:
# Step 2: Define which accessible dataset to download
dataset_accessible = 'on-street-parking-bays'

# Fetch and load the accessible parking bays dataset
df_bays = fetch_parking_dataset(
    base_url=BASE_URL,
    dataset_id=dataset_accessible,
    api_key=API_KEY,
    limit=100,               # Max allowed per API
    max_pages=100            # Adjust based on size of dataset
)

# Print the number of accessible parking bays retrieved
print("Accessible Bays Loaded:", df_bays.shape)

# Display a preview of the data
df_bays.head()


Accessible Bays Loaded: (10000, 7)


,roadsegmentid,kerbsideid,roadsegmentdescription,latitude,longitude,lastupdated,location
0,23322,None,Docklands Drive between Docklands Drive and We...,-37.816019,144.935552,2023-10-31,"{'lon': 144.9355523, 'lat': -37.8160191}"
1,22124,None,Hartley Street between Lorimer Street and West...,-37.824862,144.940270,2023-10-31,"{'lon': 144.9402698, 'lat': -37.8248623}"
2,22124,None,Hartley Street between Lorimer Street and West...,-37.824362,144.940408,2023-10-31,"{'lon': 144.9404083, 'lat': -37.8243625}"
3,22124,None,Hartley Street between Lorimer Street and West...,-37.824469,144.940379,2023-10-31,"{'lon': 144.9403789, 'lat': -37.8244685}"
4,22124,None,Hartley Street between Lorimer Street and West...,-37.824325,144.940188,2023-10-31,"{'lon': 144.9401876, 'lat': -37.8243252}"


This confirms that 10,000 accessible parking bay records were successfully loaded from the dataset. The preview shows key columns including street names, coordinates, and update dates, confirming the dataset is ready for spatial processing.


## Step 3: Prepare Parking Bay Data for Mapping

This step keeps only the rows that have valid latitude and longitude values. These are the coordinates we need to show each bay on a map.


In [43]:
# Only keep rows that have both latitude and longitude values
df_bays = df_bays[df_bays['latitude'].notnull() & df_bays['longitude'].notnull()].copy()

# Convert coordinates to float just to be sure
df_bays['latitude'] = df_bays['latitude'].astype(float)
df_bays['longitude'] = df_bays['longitude'].astype(float)

# Preview just a few key columns
df_bays[['roadsegmentdescription', 'latitude', 'longitude']].head()


,roadsegmentdescription,latitude,longitude
0,Docklands Drive between Docklands Drive and We...,-37.816019,144.935552
1,Hartley Street between Lorimer Street and West...,-37.824862,144.940270
2,Hartley Street between Lorimer Street and West...,-37.824362,144.940408
3,Hartley Street between Lorimer Street and West...,-37.824469,144.940379
4,Hartley Street between Lorimer Street and West...,-37.824325,144.940188


This output confirms that only valid coordinate data is retained for mapping. By removing null values and converting to float, we ensure clean input for accurate geospatial plotting on the map.


## Step 4: Show Accessible Parking Bays on a Map

This step displays an interactive map of Melbourne with a marker for each on-street parking bay. 

When you click a marker, you’ll see the location name and road segment info in a popup. This makes it easy to visually explore where accessible parking is located across the city.


In [ ]:
# Create a base map centered on Melbourne
m = folium.Map(location=[-37.8136, 144.9631], zoom_start=14, tiles="CartoDB positron")

# Add clustering so overlapping markers are grouped nicely
marker_cluster = MarkerCluster().add_to(m)

# Add each bay to the map
for _, row in df_bays.iterrows():
    popup_info = f"""
    <strong>Location:</strong> {row['roadsegmentdescription']}<br>
    <strong>Last Updated:</strong> {row['lastupdated']}
    """
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup_info, max_width=250),
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(marker_cluster)

# Show the map
m


This interactive map displays each accessible parking bay in Melbourne using a clustered marker. Clicking a marker reveals the street and last update date. The marker clustering improves performance and usability at different zoom levels.




# RT ACCESSIBLE PARKING – 1.2

### Mapping Off-Street Car Parks with Accessibility Info

This visualisation shows all known off-street car parks in Melbourne — including whether they have accessible bays, how many total spaces they offer, and when they’re open.

Each car park is shown as a marker on the map. You can click each marker to see important details like the number of accessible bays, total capacity, and street address.

This helps highlight which facilities offer inclusive access and where improvements might be needed.


## Step 1: Load Off-Street Car Park Data from the City of Melbourne

This step downloads all off-street car park locations in Melbourne — including their capacity, opening hours, and accessible parking features.


In [ ]:
# Step 1: Define which off-street car park dataset to download
dataset_offstreet = 'off-street-car-parks-with-capacity-and-type'

# Fetch and load the off-street car park dataset
df_offstreet = fetch_parking_dataset(
    base_url=BASE_URL,
    dataset_id=dataset_offstreet,
    api_key=API_KEY,
    limit=100,             # Max allowed per API
    max_pages=100          # Adjust based on dataset size
)

# Print the number of off-street car parks retrieved
print("Off-Street Car Parks Loaded:", df_offstreet.shape)

# Display a preview of the data
df_offstreet.head()


This confirms that 10,000 off-street car park records were successfully retrieved. The preview shows details like address, parking type (residential or private), number of parking spaces, and location coordinates. This gives a clear view of structured off-street parking supply in Melbourne.


## Step 2: Prepare Off-Street Car Park Data for Mapping

This step keeps only the rows with valid location data (latitude and longitude), and selects the most important columns like address, type, and number of parking spaces.

We also make sure the coordinates are stored as numbers (floats) so they work correctly on the map.


In [ ]:
# Remove any entries without valid coordinates
df_offstreet = df_offstreet[df_offstreet['latitude'].notnull() & df_offstreet['longitude'].notnull()].copy()

# Convert to float (just in case they're stored as text)
df_offstreet['latitude'] = df_offstreet['latitude'].astype(float)
df_offstreet['longitude'] = df_offstreet['longitude'].astype(float)

# Optional: Keep only the columns we care about
df_map = df_offstreet[[
    'building_address',
    'parking_type',
    'parking_spaces',
    'latitude',
    'longitude'
]]

# Preview cleaned data
df_map.head()


This confirms that only records with valid latitude and longitude values were kept.  
The preview shows address, type of car park, and number of spaces alongside coordinates,  
meaning the dataset is now ready for mapping.


## Step 3: Show Off-Street Car Parks on a Map

This step displays all off-street car parks on a Melbourne map. Each car park is marked with a pin, and clicking the pin shows useful details like the street address, number of parking spaces, and whether it’s private or residential.


In [ ]:
# Create a map centred on Melbourne
m = folium.Map(location=[-37.8136, 144.9631], zoom_start=14, tiles="CartoDB positron")

# Add clustering for markers
marker_cluster = MarkerCluster().add_to(m)

# Add a marker for each car park
for _, row in df_map.iterrows():
    popup = f"""
    <strong>Address:</strong> {row['building_address']}<br>
    <strong>Type:</strong> {row['parking_type']}<br>
    <strong>Total Spaces:</strong> {row['parking_spaces']}
    """
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup, max_width=250),
        icon=folium.Icon(color='green', icon='parking', prefix='fa')
    ).add_to(marker_cluster)

# Show the map
m


This interactive map displays all off-street car parks across Melbourne.  
Each marker shows details such as address, type of facility, and available spaces.  
Clustering makes the map easier to explore without overwhelming the view.


# RT ACCESSIBLE PARKING – 1.3

### Mapping Parking Meter Locations and Payment Options

This visualisation shows where on-street parking meters are located across the City of Melbourne.

Each marker represents a meter, with pop-up information about its location and what kinds of payment it accepts (e.g. credit card or Tap & Go). This helps identify areas where digital or cashless payment options are available — which can be important for planning accessible and inclusive parking.

The data comes from the City of Melbourne’s open dataset on on-street parking meters.


## Step 1: Load On-Street Parking Meter Data from the City of Melbourne

This dataset contains the locations of parking meters across Melbourne. Each record includes the zone, meter ID, and whether it is single or multi-bay.


In [ ]:
# Step 1: Define which parking meter dataset to download
dataset_meters = 'on-street-car-parking-meters-with-location'

# Fetch and load the parking meter dataset
df_meters = fetch_parking_dataset(
    base_url=BASE_URL,
    dataset_id=dataset_meters,
    api_key=API_KEY,
    limit=100,             # Max allowed per API
    max_pages=100          # Adjust based on dataset size
)

# Print the number of parking meter records retrieved
print("Parking Meter Records Loaded:", df_meters.shape)

# Display a preview of the data
df_meters.head()


This confirms that 1,296 parking meter records were successfully retrieved.  
The preview shows fields such as meter ID, location description, and payment options (e.g. card or tap-and-go).  
This gives a snapshot of how parking payments are distributed across Melbourne.


## Step 2: Prepare Parking Meter Data for Mapping

We’ll filter out any records missing location data, make sure latitude and longitude are numeric, and keep key columns like zone ID and meter type. This will allow us to show each parking meter clearly on the map.


In [ ]:
# Filter rows with valid coordinates
df_meters = df_meters[df_meters['latitude'].notnull() & df_meters['longitude'].notnull()].copy()

# Convert coordinates to float
df_meters['latitude'] = df_meters['latitude'].astype(float)
df_meters['longitude'] = df_meters['longitude'].astype(float)

# Keep relevant fields for the map
df_map_meters = df_meters[[
    'meter_id',
    'location_description',
    'creditcard',
    'tapandgo',
    'latitude',
    'longitude'
]]

# Preview cleaned data
df_map_meters.head()


This confirms that only parking meter records with valid coordinates were kept.  
The preview shows meter ID, location details, and payment methods alongside latitude and longitude.  
The dataset is now clean and ready to be mapped.


## Step 3: Show On-Street Parking Meters on a Map

This step displays all on-street parking meters across Melbourne. Each marker shows the meter ID, the zone it belongs to, and the type of bay (e.g. single or multi-bay).


In [ ]:
# Create base map
m = folium.Map(location=[-37.8136, 144.9631], zoom_start=14, tiles="CartoDB positron")
marker_cluster = MarkerCluster().add_to(m)

# Add each meter to the map
for _, row in df_map_meters.iterrows():
    popup = f"""
    <strong>Meter ID:</strong> {row['meter_id']}<br>
    <strong>Location:</strong> {row['location_description']}<br>
    <strong>Credit Card:</strong> {row['creditcard']}<br>
    <strong>Tap & Go:</strong> {row['tapandgo']}
    """
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup, max_width=250),
        icon=folium.Icon(color='orange', icon='credit-card', prefix='fa')
    ).add_to(marker_cluster)

m


This interactive map displays parking meter locations across Melbourne.  
Each clustered marker shows meter ID and payment options when clicked.  
Clustering improves readability and makes dense areas easier to explore.


# RT ACCESSIBLE PARKING – 1.4

### Visualising Parking Restrictions Across Melbourne Streets

This section shows a heatmap of all on-street parking bays, based on how restrictive they are.

Each bay includes details such as time limits, restrictions, and enforcement rules. The more restrictive the parking bay (e.g. short time limits, tow-away zones), the stronger the colour appears on the map.

This helps highlight areas where it may be harder to find flexible or accessible parking — supporting better decision-making for urban planning and disability access improvements.


## Step 1: Load On-Street Parking Bay Data from the City of Melbourne

This dataset includes every on-street parking bay in Melbourne, with information about restrictions, time limits, and location.

We’ll use it to show which areas are more restrictive than others — for example, shorter maximum stay or strict tow-away zones.


In [ ]:
# Step 1: Define which parking bay restriction dataset to download
dataset_restrictions = 'on-street-parking-bays'

# Fetch and load the parking bay restriction dataset
df_restrict = fetch_parking_dataset(
    base_url=BASE_URL,
    dataset_id=dataset_restrictions,
    api_key=API_KEY,
    limit=100,             # Max allowed per API
    max_pages=100          # Adjust based on dataset size
)

# Print the number of parking bays retrieved
print("Parking Bays Loaded:", df_restrict.shape)

# Display a preview of the data
df_restrict.head()


This confirms that 10,000 parking bay restriction records were successfully retrieved.  
The preview shows road segment IDs, descriptions, coordinates, and last updated dates.  
This gives the base dataset for analysing on-street parking restrictions.


## Step 2: Clean and Prepare Data for Heatmap

This step filters out bays without coordinates, converts them to numeric values, and focuses on key columns like restriction type and maximum stay time. 

We’ll use this to visualise restriction intensity later — for example, short stay bays can be considered more restrictive.


In [ ]:
# Keep only rows with valid coordinates
df_restrict = df_restrict[df_restrict['latitude'].notnull() & df_restrict['longitude'].notnull()].copy()

# Convert to float
df_restrict['latitude'] = df_restrict['latitude'].astype(float)
df_restrict['longitude'] = df_restrict['longitude'].astype(float)

# Keep only coordinates
df_heatmap = df_restrict[['latitude', 'longitude']]
df_heatmap.head()


This confirms that only records with valid latitude and longitude were kept.  
The preview now shows clean numeric coordinates ready for mapping.  
The dataset is prepared for use in the heatmap visualisation.


## Step 3: Visualise Parking Bay Density with a Heatmap

This map shows where on-street parking bays are concentrated across Melbourne.

The more intense the colour, the more bays exist in that area. This can help highlight areas with heavy parking control or potential congestion — useful when planning accessible or flexible parking spaces.


In [ ]:

# Create a base map centred on Melbourne
m = folium.Map(location=[-37.8136, 144.9631], zoom_start=14, tiles="CartoDB positron")

# Convert DataFrame to a list of [lat, lon] pairs
heat_data = df_heatmap[['latitude', 'longitude']].values.tolist()

# Add heatmap layer
HeatMap(heat_data, radius=9, blur=15, max_zoom=13).add_to(m)

# Display the map
m


This interactive heatmap shows where parking bays are most concentrated across Melbourne.  
Areas with brighter colours have more bays, while darker areas have fewer.  
It provides a clear picture of parking restriction intensity by location.


# RT ACCESSIBLE PARKING – 1.5

### Tracking the Growth of Accessible Parking Bays Over Time

This visualisation shows how the number of accessible (disability) parking bays has changed over time in Melbourne.

By analysing when bays were added or updated, we can track improvements in inclusive access across the city. This helps highlight periods of investment, areas of neglect, or momentum toward accessibility targets.

The data comes from the City of Melbourne's open dataset for on-street accessible bays.


## Step 1: Check Date Columns in Accessible Bays Dataset

To create a time series chart, we need to know **when** each bay was created or last updated.

Let’s see what date-related columns are available in the dataset.


In [ ]:
# Preview column names
df_bays.columns.tolist()

# Preview a few rows to find date fields
df_bays.head()


This preview confirms the dataset structure and shows the key date field `lastupdated`.  
This field will be converted to proper datetime format for time-series analysis.


## Step 2: Prepare Date Column and Group Bays by Month

This step converts the 'lastupdated' column to a proper date format, then groups the data to show how many bays were added or changed over time.


In [ ]:
# Convert lastupdated to datetime
df_bays['lastupdated'] = pd.to_datetime(df_bays['lastupdated'], errors='coerce')

# Drop rows with invalid dates
df_bays = df_bays[df_bays['lastupdated'].notnull()].copy()

# Group by month and count entries
monthly_counts = df_bays.groupby(df_bays['lastupdated'].dt.to_period('M')).size()

# Convert PeriodIndex to datetime for plotting
monthly_counts.index = monthly_counts.index.to_timestamp()

# Preview
monthly_counts.head()


This confirms that the `lastupdated` column was converted successfully into datetime format.  
The output shows the number of bay records grouped by month, ready for visualisation.


## Step 3: Plot Monthly Growth of Accessible Parking Bays

This line chart shows how many accessible bays were added (or last updated) each month.

It helps highlight when improvements were made and whether growth has increased, slowed, or remained steady over time.


In [ ]:
# Plotting
plt.figure(figsize=(12, 6))
monthly_counts.plot(kind='line', marker='o', linewidth=2)

# Labels and title
plt.title('Monthly Accessible Parking Bay Updates in Melbourne', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Number of Bays Updated', fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.show()


This line chart shows how the number of bay records changes over time.  
It highlights periods where new data was added or updated more frequently.


# RT ACCESSIBLE PARKING – 1.6P

### Comparing Accessibility Coverage with High-Demand Parking Zones

This visualisation overlays two layers:

1. Accessible parking bay locations  
2. Parking meter density (as a proxy for parking demand)

By combining these, we can identify areas where demand is high but accessible infrastructure is missing — helping planners prioritise improvements where they’re most needed.


In [ ]:
# Accessible bays (from df_bays used earlier)
accessible_coords = df_bays[['latitude', 'longitude']].dropna()
accessible_coords = accessible_coords.astype(float)

# Parking meters (from df_map_meters or df_meters)
meter_coords = df_meters[['latitude', 'longitude']].dropna()
meter_coords = meter_coords.astype(float)

print("Accessible Bays:", accessible_coords.shape[0])
print("Parking Meters:", meter_coords.shape[0])


## Step 2: Display Multi-Layer Map of Accessibility and Demand

This map shows two layers:

- **Blue markers**: accessible parking bays (individual markers)
- **Red heatmap**: areas with a high density of parking meters (demand zones)

This lets us visually identify areas with high parking activity but low accessible bay coverage.


In [ ]:
# Create base map
m = folium.Map(location=[-37.8136, 144.9631], zoom_start=14, tiles="CartoDB positron")

# Add demand heatmap (meter density)
heat_data = meter_coords[['latitude', 'longitude']].values.tolist()
HeatMap(heat_data, radius=10, blur=15, gradient={0.4: 'orange', 0.7: 'red'}).add_to(m)

# Add accessible bay markers
for _, row in accessible_coords.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        color='blue',
        fill=True,
        fill_opacity=0.8
    ).add_to(m)

# Show map
m


## RT ACCESSIBLE PARKING – 1.7P

### Exploratory Analysis of Parking Distribution

This section provides an early look at how parking bays are distributed across the City of Melbourne. This helps to contextualise where accessibility features are concentrated or lacking.


In [ ]:
# Count the number of bays per road segment
df_bays['roadsegmentdescription'].value_counts()


This breakdown shows how parking bays are distributed across road segments in Melbourne. It helps identify which streets have the highest concentration of bays.


In [ ]:
# Compare segments across different zones
segment_counts = (
    df_bays['roadsegmentdescription']
      .value_counts()
      .rename_axis('roadsegment')
      .to_frame('bay_count')
)
segment_counts.head()



This table shows the number of accessible bays per road segment.  
It highlights which streets have the highest concentration of disability bays.


In [ ]:
# Count bays per kerbside ID (sub-segment of roads)
df_bays['kerbsideid'].value_counts()


This count reveals how many bays are located at each kerbside segment. It gives a more detailed look at the parking layout within road segments.


In [ ]:
# Get top 10 most common road segments
top_segments = df_bays['roadsegmentdescription'].value_counts().head(10)

# Create horizontal bar plot
plt.figure(figsize=(10, 6))
top_segments.plot(kind='barh', color='teal')
plt.xlabel('Number of Bays')
plt.ylabel('Road Segment')
plt.title('Top 10 Road Segments with Most Parking Bays')
plt.gca().invert_yaxis()  # optional: highest bar on top
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


This horizontal bar chart visualises the 10 road segments with the highest number of parking bays. It provides a clearer view of street-level accessibility distribution.


IpyWidgets to come, need to split into part1 and part 2


## Step 1: Create an IpyWidgets app layout

This sets up the app structure using tabs, so users can switch between each visual - unfinished.


In [ ]:
# Create empty output widgets for each tab
tab1_out = Output()
tab2_out = Output()
tab3_out = Output()
tab4_out = Output()
tab5_out = Output()
tab6_out = Output()

# Define tabs and assign titles
tabs = Tab(children=[tab1_out, tab2_out, tab3_out, tab4_out, tab5_out, tab6_out])
tabs.set_title(0, "1.1P Accessible Bays")
tabs.set_title(1, "1.2P Off-Street Car Parks")
tabs.set_title(2, "1.3P Parking Meters")
tabs.set_title(3, "1.4P Restriction Heatmap")
tabs.set_title(4, "1.5P Bays Over Time")
tabs.set_title(5, "1.6P Accessibility vs Demand")

# Display the dashboard title and the tabs
display(HTML("<h1> RT ACCESSIBLE PARKING Dashboard</h1>"))
display(HTML("<p>This dashboard visualises accessible parking across Melbourne using open data.</p>"))
display(tabs)


In [ ]:
with tab1_out:
    display(HTML("<h3>1.1P Accessible Bays</h3>"))

    # Example map 
    bay_map = folium.Map(location=[-37.8136, 144.9631], zoom_start=13)
    folium.Marker(location=[-37.8136, 144.9631], popup="Example Bay").add_to(bay_map)

    display(bay_map)


In [ ]:
with tab2_out:
    display(HTML("<h3>1.2P Off-Street Car Parks</h3>"))

    carpark_map = folium.Map(location=[-37.8136, 144.9631], zoom_start=13)

    # Example markers 
    folium.Marker(location=[-37.81, 144.96], popup="Car Park A").add_to(carpark_map)
    folium.Marker(location=[-37.82, 144.95], popup="Car Park B").add_to(carpark_map)

    display(carpark_map)


In [ ]:
with tab3_out:
    display(HTML("<h3>1.3P Parking Meters</h3>"))

    meter_map = folium.Map(location=[-37.8136, 144.9631], zoom_start=13)

    # Example marker 
    folium.Marker(location=[-37.814, 144.965], popup="Meter 001").add_to(meter_map)

    display(meter_map)


In [ ]:
with tab4_out:
    display(HTML("<h3>1.4P Restriction Heatmap</h3>"))

    heat_map = folium.Map(location=[-37.8136, 144.9631], zoom_start=13)

    # Example coordinates 
    heat_data = [[-37.8136, 144.9631], [-37.814, 144.9645], [-37.815, 144.962]]
    HeatMap(heat_data).add_to(heat_map)

    display(heat_map)


In [ ]:
with tab5_out:
    display(HTML("<h3>1.5P Bays Over Time</h3>"))

    # Example matplotlib plot — replace with your own df timeline analysis
    import matplotlib.pyplot as plt

    fig, ax = plt.subplots()
    ax.plot([1, 2, 3, 4], [100, 130, 150, 180], marker='o')
    ax.set_title("Accessible Bays Added Over Time")
    ax.set_xlabel("Year")
    ax.set_ylabel("Bays")

    display(fig)
    

In [ ]:
with tab6_out:
    display(HTML("<h3>1.6P Accessibility vs Demand</h3>"))

    # Example bar chart (static) — replace with real accessibility vs usage data
    fig, ax = plt.subplots()
    categories = ['CBD', 'Docklands', 'Carlton', 'Southbank']
    access = [120, 80, 95, 60]
    demand = [200, 150, 100, 90]

    ax.bar(categories, access, label='Accessible Bays', alpha=0.7)
    ax.bar(categories, demand, label='Estimated Demand', alpha=0.7)
    ax.set_title("Accessibility vs Estimated Demand")
    ax.legend()

    display(fig)
